## Make the data all of the same length

In [1]:
import os
import cv2
import shutil
import pandas as pd
import numpy as np

# Load in length of each data point
## Not useful as some coordinate data do not match video lengths

In [ ]:
vidLength = {}
f = open("videoLength.txt", "r")
line = f.readline()
while (line):
    try:
        if line.split("_|_")[1] == 'None\n':
            vidLength[line.split("_|_")[0]] = None
        else:
            vidLength[line.split("_|_")[0]] = int(line.split("_|_")[1])
    except:
        if line != '\n':
            print(line)
    line = f.readline()
f.close()
vidLength

In [3]:
videos =  list(vidLength.items())

maxLength = max(videos)
minLength = max(videos)

print("Longest video length: ", max(videos))
print("Shortest video length: ", min(videos))

Longest video length:  ('ZOOM-OFF_STBA1c2b_36540_37860.mp4', 33)
Shortest video length:  ('ABOUT1_BFSA1c2a_1630_1990.mp4', 9)


# Load in Coordinate data

In [ ]:
labelLocation = 'D:/Thesis/ELAR-Data/label.csv'
labelDf = pd.read_csv(labelLocation)
labelDict = dict(np.array(labelDf))
print(labelDict)

X = {}
coorDataLocation = "D:/Thesis/ELAR-Data/arrayData/"
dirData = os.listdir(coorDataLocation)
failed = 0
print(dirData)
for i, data in enumerate(dirData):
    if 'shape' in data:
        continue
    fileLocation = coorDataLocation + data
    shapeLocation = coorDataLocation + dirData[i+1]
    coorLoad = np.loadtxt(fileLocation)
    coorShape= np.loadtxt(shapeLocation)
    try:
        coorLoad = coorLoad.reshape(coorLoad.shape[0], coorLoad.shape[1] // int(coorShape[2]), int(coorShape[2]))
        X[data[:-4]] = coorLoad
    except IndexError:
        print(failed, fileLocation, shapeLocation)
        failed += 1

coorData = X
# coorData = {Y[i]:x for i, x in enumerate(X)}

# Load in length of each data point

In [6]:
vidLength = {key: data.shape[0] for key, data in coorData.items()}
len(vidLength)

14096

In [8]:
videos =  list(vidLength.items())

maxLength = max(videos)
minLength = max(videos)

print("Longest video length: ", max(videos))
print("Shortest video length: ", min(videos))

Longest video length:  ('ZOOM-OFF_STBA1c2b_36540_37860', 33)
Shortest video length:  ('ABOUT1_SBS1A3c7a_950_1280', 9)


# Resize Technique
### Note the coordinate data is in a 3d array (i,j,k) where the
- Where i is the frame
- Where j is the 33 different joints
- k is the x y z of that specific joing
- e.g (22, 33, 3) could be the shape for a data point

In [ ]:
eg = "ABOUT1_BFSA1c2a_1630_1990"
coordinate = coorData[eg]
coordinate.shape

In [10]:
resizeLength = 40

In [ ]:
newCoorDic = {}
# Impute the mean
for name, originalLength in videos:
    coordinate = coorData[name]
    imputeEvery = originalLength/resizeLength
    i = 0
    # List of all new Joint location
    newCoors = [None] * resizeLength
    for newFrame in range(resizeLength):
        # new Joint location for a given frame
        frameCoors = [None] * 33

        # new frame based between these frames
        baseFrame = int(np.floor(imputeEvery * newFrame))
        nextFrame = int(baseFrame + 1)
        if nextFrame >= originalLength:
            nextFrame = baseFrame
        
        # Bias when averaging frames
        bias =  1 - ((imputeEvery * newFrame) - baseFrame)
        for i in range(33):
            xyz = [None] * 3
            for j in range(3):
                newjoint = ((1 + bias) * coordinate[baseFrame][i][j] 
                            + (1 - bias) * coordinate[nextFrame][i][j])/2
                xyz[j] = newjoint
            frameCoors[i] = xyz
        newCoors[newFrame] = frameCoors
    newCoorDic[name] = newCoors
newCoorDic
            

In [1]:
# PsuedoCode
newCoordinates = []
for i from 1 to resizeLength
    baseFrame = floor(ImputationRatio * i) # 



imputationRatio = originalLength/resizeLength

In [12]:
reshaped_data =  np.array(list(newCoorDic.values()))
reshaped_data.shape

(14096, 40, 33, 3)

# Save new Frames

In [ ]:
coor = []
for name, coor in newCoorDic.items():
    if len(coor) > 0:
        coor = np.array(coor)
        print("saving: ", name)
        np.savetxt("D:/Thesis/ELAR-Data/heavyImputedArrayData/"+ name +".npy", coor.reshape(coor.shape[0], -1))
        np.savetxt("D:/Thesis/ELAR-Data/heavyImputedArrayData/"+ name +"_shape.npy", np.array(coor.shape))